# Spatio-Temporal Stage2 モデル

交通事故データを用いた「空間・時系列（Spatio-Temporal）を取り入れた死亡事故予測モデル」の実装と評価。

## 目次
1. 環境設定
2. データ読み込みと前処理
3. グラフ構築
4. モデル学習
5. 評価
6. 可視化（地図ヒートマップ）

## 1. 環境設定

In [ ]:
import sys
import os
from pathlib import Path

# パス設定
PROJECT_ROOT = Path.cwd().parent.parent
sys.path.insert(0, str(PROJECT_ROOT / 'scripts' / 'spatio_temporal'))

print(f"Project Root: {PROJECT_ROOT}")

In [ ]:
# 必要なパッケージのインポート
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns

# 自作モジュール
from preprocess_spatio_temporal import SpatioTemporalPreprocessor
from graph_builder import GraphBuilder, build_sample_graph
from train_spatio_temporal import SpatioTemporalTrainer
from evaluate import evaluate_model, ModelEvaluator
from visualize import plot_pr_curve, plot_roc_curve, create_heatmap
from utils.checkpoint import set_seed

# ランダムシード固定
RANDOM_SEED = 42
set_seed(RANDOM_SEED)

# デバイス
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

## 2. データ読み込みと前処理

In [ ]:
# データパス
DATA_PATH = PROJECT_ROOT / 'data' / 'processed' / 'honhyo_for_analysis_with_traffic_hospital_no_leakage.csv'
OUTPUT_DIR = PROJECT_ROOT / 'data' / 'spatio_temporal'

print(f"Data Path: {DATA_PATH}")
print(f"Exists: {DATA_PATH.exists()}")

In [ ]:
# 前処理の実行
preprocessor = SpatioTemporalPreprocessor(
    data_path=str(DATA_PATH),
    output_dir=str(OUTPUT_DIR),
    train_years=(2018, 2019),
    val_years=(2020, 2020),
    test_years=(2021, 2024),
    geohash_precision=6,
)

# 前処理実行（または既存ファイルを読み込み）
if not (OUTPUT_DIR / 'preprocessed_train.parquet').exists():
    preprocess_result = preprocessor.run()
    print(f"\n前処理結果: {preprocess_result}")
else:
    print("前処理済みファイルが存在します。スキップ。")

In [ ]:
# 前処理済みデータの読み込み
train_df = pd.read_parquet(OUTPUT_DIR / 'preprocessed_train.parquet')
val_df = pd.read_parquet(OUTPUT_DIR / 'preprocessed_val.parquet')
test_df = pd.read_parquet(OUTPUT_DIR / 'preprocessed_test.parquet')

print(f"Train: {len(train_df):,} rows")
print(f"Val:   {len(val_df):,} rows")
print(f"Test:  {len(test_df):,} rows")

# ターゲット分布
print(f"\nTrain Fatal: {train_df['fatal'].sum():,} ({train_df['fatal'].mean()*100:.2f}%)")
print(f"Val Fatal:   {val_df['fatal'].sum():,} ({val_df['fatal'].mean()*100:.2f}%)")
print(f"Test Fatal:  {test_df['fatal'].sum():,} ({test_df['fatal'].mean()*100:.2f}%)")

In [ ]:
# データの確認
train_df.head()

## 3. グラフ構築

In [ ]:
# kNNグラフの構築
k = 8

builder = GraphBuilder(k=k, use_haversine=True)

# 学習データの座標
coords = train_df[['lat', 'lon']].values
print(f"座標サンプル数: {len(coords):,}")

# グラフ構築（サブサンプリング）
sample_size = min(10000, len(coords))
sample_idx = np.random.choice(len(coords), sample_size, replace=False)
sample_coords = coords[sample_idx]

edge_index, edge_attr = builder.build_knn_graph(sample_coords)

print(f"\nサンプルグラフ:")
print(f"  ノード数: {sample_size:,}")
print(f"  エッジ数: {edge_index.shape[1]:,}")
print(f"  平均次数: {edge_index.shape[1] / sample_size:.2f}")

In [ ]:
# エッジ距離の分布
if edge_attr is not None:
    distances = edge_attr.numpy().flatten()
    
    plt.figure(figsize=(10, 5))
    plt.hist(distances, bins=50, edgecolor='black', alpha=0.7)
    plt.xlabel('Distance (km)')
    plt.ylabel('Frequency')
    plt.title('kNN Edge Distance Distribution')
    plt.axvline(np.median(distances), color='r', linestyle='--', label=f'Median: {np.median(distances):.2f} km')
    plt.legend()
    plt.tight_layout()
    plt.show()

## 4. モデル学習

In [ ]:
# モデル設定
config = {
    'hidden_dim': 128,
    'num_layers': 2,
    'dropout': 0.3,
    'learning_rate': 0.001,
    'batch_size': 1024,
    'epochs': 50,  # Notebookでは短めに
    'patience': 10,
    'focal_alpha': 0.75,
    'focal_gamma': 2.0,
    'k_neighbors': 8,
}

print("モデル設定:")
for k, v in config.items():
    print(f"  {k}: {v}")

In [ ]:
# 学習実行
RESULTS_DIR = PROJECT_ROOT / 'results' / 'spatio_temporal'

trainer = SpatioTemporalTrainer(
    data_dir=str(OUTPUT_DIR),
    output_dir=str(RESULTS_DIR),
    model_type='mlp',  # まずMLPベースライン
    config=config,
)

results = trainer.run()

## 5. 評価

In [ ]:
# テスト予測結果の読み込み
test_pred_path = RESULTS_DIR / 'test_predictions.parquet'

if test_pred_path.exists():
    test_pred_df = pd.read_parquet(test_pred_path)
    print(f"テスト予測: {len(test_pred_df):,} rows")
    
    # 評価
    y_true = test_pred_df['fatal'].values
    y_pred = test_pred_df['prediction'].values
    
    metrics = evaluate_model(y_true, y_pred)
    
    print("\n評価結果:")
    for k, v in metrics.items():
        if isinstance(v, float):
            print(f"  {k}: {v:.4f}")
else:
    print("テスト予測ファイルが見つかりません")

In [ ]:
# PR曲線
from sklearn.metrics import precision_recall_curve, auc

if 'y_true' in dir() and 'y_pred' in dir():
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)
    pr_auc = auc(recall, precision)
    
    plt.figure(figsize=(10, 8))
    plt.plot(recall, precision, 'b-', lw=2, label=f'PR-AUC = {pr_auc:.4f}')
    plt.axhline(y=y_true.mean(), color='gray', linestyle='--', label=f'Baseline = {y_true.mean():.4f}')
    plt.xlabel('Recall', fontsize=12)
    plt.ylabel('Precision', fontsize=12)
    plt.title('Precision-Recall Curve', fontsize=14)
    plt.legend(loc='best')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# 予測確率の分布
if 'y_pred' in dir():
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 全体分布
    axes[0].hist(y_pred, bins=50, edgecolor='black', alpha=0.7)
    axes[0].set_xlabel('Predicted Probability')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('Prediction Distribution (All)')
    
    # クラス別分布
    axes[1].hist(y_pred[y_true == 0], bins=50, alpha=0.5, label='Negative')
    axes[1].hist(y_pred[y_true == 1], bins=50, alpha=0.5, label='Positive (Fatal)')
    axes[1].set_xlabel('Predicted Probability')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Prediction Distribution by Class')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()

## 6. 可視化（地図ヒートマップ）

In [ ]:
# 高リスク地点の確認
if 'test_pred_df' in dir():
    top_n = 100
    high_risk = test_pred_df.nlargest(top_n, 'prediction')
    
    print(f"Top-{top_n} 高リスク地点:")
    print(f"  予測確率: {high_risk['prediction'].min():.4f} - {high_risk['prediction'].max():.4f}")
    print(f"  実際の死亡事故: {high_risk['fatal'].sum()}/{top_n}")
    print(f"  Precision@{top_n}: {high_risk['fatal'].mean():.4f}")

In [ ]:
# ヒートマップ生成
if 'test_pred_df' in dir():
    heatmap_path = RESULTS_DIR / 'heatmap_notebook.html'
    
    # 座標が有効なサンプルのみ使用
    valid_df = test_pred_df.dropna(subset=['lat', 'lon', 'prediction'])
    
    if len(valid_df) > 0:
        create_heatmap(
            valid_df,
            str(heatmap_path),
            title='交通事故死亡リスク予測ヒートマップ'
        )
        print(f"\nヒートマップ保存: {heatmap_path}")
        print("ブラウザで開いて確認してください。")
    else:
        print("有効な座標データがありません")

In [ ]:
# 地図をNotebook内で表示（foliumがインストールされている場合）
try:
    import folium
    from folium.plugins import HeatMap
    
    if 'valid_df' in dir() and len(valid_df) > 0:
        # サンプリング（表示用）
        sample_df = valid_df.sample(min(5000, len(valid_df)))
        
        # 地図作成
        center_lat = sample_df['lat'].mean()
        center_lon = sample_df['lon'].mean()
        
        m = folium.Map(location=[center_lat, center_lon], zoom_start=6)
        
        heat_data = [[row['lat'], row['lon'], row['prediction']] 
                     for _, row in sample_df.iterrows()]
        
        HeatMap(heat_data, radius=15, blur=10).add_to(m)
        
        # Notebook内で表示
        display(m)
except ImportError:
    print("foliumがインストールされていません。pip install folium でインストールしてください。")

## まとめ

このNotebookでは以下を実行しました：

1. **データ前処理**: ジオハッシュ生成、時系列特徴量、時間ベース分割
2. **グラフ構築**: kNNグラフ（Haversine距離）
3. **モデル学習**: MLP（ベースライン）
4. **評価**: PR-AUC, ROC-AUC, Precision/Recall@k
5. **可視化**: ヒートマップ、PR曲線

### 次のステップ

- kNN-GNNモデルの学習と比較
- Optunaによるハイパーパラメータ探索
- Temporal GNNの実装

### 実行コマンド

```bash
# 全工程を一括実行
python scripts/spatio_temporal/run.py --all

# Optuna探索付き
python scripts/spatio_temporal/run.py --all --optuna
```